In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
data = pd.read_csv("heart_disease_dataset.csv") #Dosyayı yükle

In [3]:
data.sample(10) #data.sample(10) ifadesi, pandas DataFrame'i olan data'dan rastgele 10 satır örneği seçmek için kullanılır. Bu yöntem, veri setinden rastgele örnekler almak ve veri keşfi, model doğrulama veya veri görselleştirme gibi işlemler için yararlıdır. Detaylı açıklama aşağıdaki gibidir:

,Age,Gender,CP,Trestbps,Chol,Fbs,Restecg,Thalach,Exang,Oldpeak,Slope,CA,Thal,Num
290,67,1,3,152,212,0,2,150,0,0.8,2,0,7,1
272,46,1,4,140,311,0,0,120,1,1.8,2,2,7,2
235,54,1,4,122,286,0,2,116,1,3.2,2,2,3,3
173,62,0,4,140,394,0,2,157,0,1.2,2,0,3,0
68,59,1,4,170,326,0,2,140,1,3.4,3,0,7,2
115,41,1,2,135,203,0,0,132,0,0.0,2,0,6,0
222,39,0,3,94,199,0,0,179,0,0.0,1,0,3,0
255,42,0,3,120,209,0,0,173,0,0.0,2,0,3,0
267,59,1,3,126,218,1,0,134,0,2.2,2,1,6,2
92,62,1,3,130,231,0,0,146,0,1.8,2,3,7,0


In [4]:
data.shape #Toplam satır ve sutun sayısı

(303, 14)

In [5]:
data.isnull().sum() #Missing data kontrolü

Age         0
Gender      0
CP          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
CA          0
Thal        0
Num         0
dtype: int64

In [6]:
data.dtypes #Verilerin tipi

Age           int64
Gender        int64
CP            int64
Trestbps      int64
Chol          int64
Fbs           int64
Restecg       int64
Thalach       int64
Exang         int64
Oldpeak     float64
Slope         int64
CA           object
Thal         object
Num           int64
dtype: object

In [7]:
df = pd.DataFrame(data) #dropna:dataframe(df) ten alınan tüm null değerlerini kaldırır.

In [8]:
indexes_to_drop = df[df.apply(lambda x: x.astype(str).str.contains('\?').any(), axis=1)].index
#x.astype(str): Satırdaki tüm değerleri string (metin) tipine çevirir.
#x.astype(str).str.contains('\?'): Satırdaki herhangi bir hücrenin '?' içerip içermediğini kontrol eder.
#.any(): Satırda '?' içeren herhangi bir hücre olup olmadığını kontrol eder. Varsa True, yoksa False döner.
#.index: Koşulu sağlayan satırların indekslerini alır ve indexes_to_drop değişkenine atar.
df_cleaned = df.drop(indexes_to_drop)

<>:1: SyntaxWarning: invalid escape sequence '\?'
<>:1: SyntaxWarning: invalid escape sequence '\?'
/var/folders/y6/npmgkj1j4mzgx950yrf632lc0000gn/T/ipykernel_992/1790719935.py:1: SyntaxWarning: invalid escape sequence '\?'
  indexes_to_drop = df[df.apply(lambda x: x.astype(str).str.contains('\?').any(), axis=1)].index


In [9]:
df_cleaned['Num'] = df_cleaned['Num'].replace([2, 3, 4], 1) 
#df_cleaned['Num']: Bu, df_cleaned DataFrame'inin 'Num' sütununu seçer.
#.replace([2, 3, 4], 1): Bu metot, 'Num' sütununda 2, 3 veya 4 olan değerleri 1 ile değiştirir.
print(df_cleaned)
#print(df_cleaned): Bu, güncellenmiş df_cleaned DataFrame'ini yazdırır.

     Age  Gender  CP  Trestbps  Chol  Fbs  Restecg  Thalach  Exang  Oldpeak  \
0     63       1   1       145   233    1        2      150      0      2.3   
1     67       1   4       160   286    0        2      108      1      1.5   
2     67       1   4       120   229    0        2      129      1      2.6   
3     37       1   3       130   250    0        0      187      0      3.5   
4     41       0   2       130   204    0        2      172      0      1.4   
..   ...     ...  ..       ...   ...  ...      ...      ...    ...      ...   
297   57       0   4       140   241    0        0      123      1      0.2   
298   45       1   1       110   264    0        0      132      0      1.2   
299   68       1   4       144   193    1        0      141      0      3.4   
300   57       1   4       130   131    0        0      115      1      1.2   
301   57       0   2       130   236    0        2      174      0      0.0   

     Slope CA Thal  Num  
0        3  0    6    0  

In [10]:
df_cleaned.Num.value_counts() # "diagnosis" feature'ı altındaki değerlerin sayısını verir

Num
0    160
1    137
Name: count, dtype: int64

In [11]:
X = df_cleaned.drop(['Num'], axis=1)
#X: df_cleaned DataFrame'inden 'Num' sütunu hariç tüm sütunları içeren yeni bir DataFrame oluşturur.
#df_cleaned.drop(['Num'], axis=1): Bu metot, df_cleaned DataFrame'inden 'Num' sütununu kaldırır.
#['Num']: Kaldırılacak sütunların listesi (bu durumda sadece 'Num' sütunu).
#axis=1: Sütunu kaldırmak için kullanılır (satırı kaldırmak için axis=0 kullanılır).
Y = df_cleaned.Num
#Y:df_cleaned DataFrame'inin 'Num' sütununu seçer ve hedef değişken olarak ayırır.


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 33) 
#X ve y verisini %70-%30 oranın train-test verisi olarak ayırır.

In [13]:
clf = MLPClassifier() #KNN Sınıflandırısı
clf.fit(X_train, Y_train) #Eğitim verilerinin, KNN algoritmasına fit edilmesi
prediction = clf.predict(X_test) #Test verisinin tahminlenme işlemi
prediction

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1])

In [14]:
accuracy_score(Y_test, prediction)

0.7111111111111111

In [15]:
cm = confusion_matrix(Y_test, prediction) # Confusion Matrix
print(cm)

[[23 22]
 [ 4 41]]
